# Herança e Composição

Herança e composição são dois conceitos fundamentais na orientação a objetos que modelam **relacionamentos** entre duas classes.

## Herança

A herança é uma estratégia para modelarmos relações do tipo **"isto é"** (is a), onde podemos definir classes especializadas com base em uma classe genérica. As classes especializadas são denominadas classes derivadas ou subclasses, enqunato as classes mais genéricas são denominadas classes base (ou superclasse).

Herança é usada principalmente para **incrementar** ou **especializar** funcionalidades de uma classe base, enquanto as demais funcionalidades permanecem iguais. Em outras palavras, podemos:

- Adicionar comportamentos
- Modificar comportamentos

Ao mesmo tempo em que os códigos definidos na classe base são automaticamente herdados pelas classes derivadas, evitando duplicidade de código.

Em Python, a sintaxe para herança é da forma `class <SubClassName>(<SuperClassName>):`

In [12]:
class Animal:
    """Classe base: Animal"""
    def walk(self):
        print("I'm an animal and I walk")

class Cachorro(Animal):
    """Classe derivada: Cachorro"""
    def play(self):
        print("I'm an dog and I like to chase balls")

In [13]:
c = Cachorro()
c.walk()

I'm an animal and I walk


In [12]:
# Diferente de instâncias da classe Animal, os da classe cachoro possuem um comportamento adicional "play" (brincar)
#
c.play()

I'm an dog and I like to chase balls


### `super()`

Para aproveitar de funcionalidades já implementadas na classe base podemos usar a função `super()`. A `super()` é uma função que retorna uma instância da classe base, permitindo-nos chamar diretamente seus métodos. O uso mais comum de `super()` é para inicializar atributos definidos na classe base (mais comum, não o único!)

> Note que na classe `Cachorro`, não é interessante replicarmos a lógica de atribução do dado `self.weight`. A melhor abordagem seria aproveitar da lógica já existente na classe base.

In [14]:
class Animal:
    
    def __init__(self, weight):
        self.weight = weight


class Cachorro(Animal):
    
    def __init__(self, weight, sound):
        super().__init__(weight)
        self.sound = sound

In [15]:
animal = Animal(10)
animal.weight

10

In [16]:
cachorro = Cachorro(10, "Au Au")
cachorro.weight, cachorro.sound

(10, 'Au Au')

### Sobrecarga de Métodos

Além de adicionar novos comportamentos à classes, podemos querer modificar comportamentos já existentes.

Por exemplo, podemos considerar que todo animal emite um som. Porém, o som de um gato é diferente de um cachorro. Logo, ambas entidades implementam um método `sound()`, mas cada método executa uma lógica diferente.

In [23]:
class Animal:
    def sound(self):
        print("I'm an animal!")

class Cachorro(Animal):
    def sound(self):
        print("Au au")

In [24]:
animal = Animal()
animal.sound()

I'm an animal!


In [25]:
cachorro = Cachorro()
cachorro.sound()

Au au


Tal estratégia de especializar comportamentos de acordo com cada subclasse é denominada polimorfismo (ad-hoc, uma vez que existem outros tipos de polimorfismo) e é utilizada com a ideia de fornecer uma interface (no caso, a assinatura do método) única com o qual diferentes elementos do código podem existir, porém o comportamento em si varia de quem interage.

Contudo, dado que Python é uma linguagem de tipagem dinâmica, pouco importa o tipo do objeto passado como argumento. Seja `Animal` ou não, contanto que implemente o método desejado, podemos invocá-lo.

Isso significa que o uso de herança continua sendo interessante para o compartilhamento de um código único entre diversas subclasses. Contudo, caso o código compartilhado seja apenas uma interface pública, podemos reconsiderar a implementação aproveitando do *duck typing*.

Por outro lado, é fato que mesmo que um objeto satisfaça uma interface em particular, isso não significa que ele irá funcionar em todas as situações.

In [26]:
class Pato:
    def sound(self):
        print("Quack quack")

In [27]:
def make_sound(animal):
    animal.sound()

In [28]:
make_sound(cachorro)

Au au


In [29]:
pato = Pato()
make_sound(pato)

Quack quack


### Classes Abstratas

Embora o *duck typing* seja útil, nem sempre é fácil ou manutenível criar classes que contém todos os comportamentos necessários para o funcionameneto do código. Nesse contexto, podemos definir interfaces de forma mais rígida, que é o caso das classes base abstratas.

Classes base abstratas (ou ABCs, do inglês Abstract Base Classes) são classes que definem um conjunto mínimo (e fundamental) de métodos (comportamentos) que uma classe deve implementar. Com isso, podemos criar classes derivadas que possuem sua própria implementação de cada funcionalidade da classe abstrata base.

Os métodos definidos pela classe abstratas que precisam ser especializados nas classes derivadas são denominados métodos abstratos. Quando uma classe só possui métodos abstratos, chamamos de classe abstrata pura.


> Note que a classe base não pode ser instanciada, uma vez que deve servir apenas como um modelo (template)
>
> Além disso, as classes derivadas **precisam** sobreescrever todos os métodos abstratos definidos nas classes base. 

In [30]:
from abc import ABC, abstractmethod

class Animal(ABC):
    def __init__(self, idade, peso):
        self.idade = idade
        self.peso = peso

    def eat(self):
        return "I'm eating!"

    @abstractmethod
    def sound(self):
        pass


class Cachorro(Animal):
    def __init__(self, idade, peso, name):
        super().__init__(idade, peso)
        self.name = name
    
    def sound(self):
        print('au au!')

In [32]:
cachorro = Cachorro(idade=1, peso=12, name='bruce')
cachorro.sound()

au au!


In [33]:
# Note que a classe abstrata não pode ser instanciada
#
animal = Animal('animal')

TypeError: Can't instantiate abstract class Animal with abstract method sound

In [34]:
class Gato(Animal):

    def __init__(self):
        super().__init__('gato')

In [35]:
# Assim como não podemos instanciar uma classe
# que não possui todos os métodos abstratos da classe base redefinidos
#
gato = Gato()

TypeError: Can't instantiate abstract class Gato with abstract method sound

### Desvantagens do uso de Herança

Herança é um mecanismo que utilizamos para modelarmos relacionamentos de **"isto é"** (por exemplo, um cachorro é um Animal, um gato é um Animal) de forma que comportamentos padrões entre todas as subclasses de uma superclasse são herdados sem necessidade de reescrita de código (evitando assim duplicação). Logo, note que o foco do uso de herança está em: organização lógica das entidades e minimização de duplicidade de código.

Contudo, a Herança possui dois problemas que devem ser evitados:

- Explosão de Classes
- Acoplamento 

#### Explosão de Classes

Dependendo de como usamos herança, podemos acabar criando uma hierarquia grande e complexa de classes.

Por exemplo, supondo um cenário onde queremos implementar um sistema de funcionários de uma empresa que possui diversos tipos de funcionários e cada funcionário possui um tipo de contrato de pagamento. Note como o número de classes cresce exponencialmente para cada variação desejada.

Este problema, conhecido como "explosão de classes", é uma das principais desvantagens no uso de herança e o principal motivo que leva muitas pessoas desenvolvedoras a odiarem herança.

In [57]:
class PessoaFuncionaria:
    pass

class PessoaProgramadora(PessoaFuncionaria):
    pass

class PessoaGerente(PessoaFuncionaria):
    pass

class PessoaVendedora(PessoaFuncionaria):
    pass

class PessoaProgramadoraHora(PessoaProgramadora):
    pass

class PessoaProgramadoraComissao(PessoaProgramadora):
    pass

class PessoaGerenteHora(PessoaGerente):
    pass

class PessoaVendedoraHora(PessoaVendedora):
    pass

class PessoaVendedoraComissao(PessoaVendedora):
    pass

Em contrapartida, podemos usar composição para facilitar as coisas, como na seção [Favorecer Composição sobre Herança](#favorecer-composicao-sobre-heranca).

#### Alto Acoplamento

> Para mais detalhes sobre acoplamento (e coesão), [clique aqui](#).

Alterações na classe base **podem** afetar as classes derivadas. Embora na maioria das situações esse acoplamento seja exatamente o comportamento desejado, as vezes isso pode ser problemático.

Por exemplo, vamos supor que temos uma classe Veículo com especializações: Carro, Motocicleta e Veículo Lunar. Qualquer comportamento comum (entre as classes derivadas), tal como alterações neste, podem ser executadas na classe `Veículo` e, todas as demais classes serão impactadas.

In [40]:
class Veiculo:
    def __init__(self):
        print("Veículo criado")
    
    def ligar(self):
        print("Veículo ligado")
    
    def trocar_pneu(self):
        # print("Não sei trocar pneu") 
        print("Pneu trocado!")  # Modificação que corrige um bug em todas as classes derivadas sem necessidade de duplicação de código

class Carro(Veiculo):
    def __init__(self):
        print("Carro criado")
        
class Motocicleta(Veiculo):
    def __init__(self):
        print("Motocicleta criada")

class VeiculoLunar(Veiculo):
    def __init__(self):
        print("Veículo Lunar criado")

Porém, e se, **posteriormente**, surgiu a necessidade de aplicar uma alteração que só é válida para uma parte das classes derivadas, como, por exemplo, trocar a vela do motor? Carros e motocicletas possuem motores a combustão com vela, mas um veículo lunar não!

Em uma base de código grande, onde muitas classes caem na mesma situação que `VeiculoLunar`, executar tal alteração vai exigir um grande esforço com muitas linhas de código impactadas. Consequentemente, a inserção de bugs é muito provável.

Uma forma de minimizar esse problema é favorecer o uso de composição em relação a herança. Logo, basta criarmos classes `MotorCombustao`, `MotorEletrico`, definir que, inicialmente, todos os veículos terão motor a combustão e sobreescrever classes cuja definição não é válida. Note que isso ainda vai gerar um grande trabalho, porém menor.

In [42]:
class MotorCombustao:
    def __init__(self):
        print('Motor à combustão criado')
    def change_spark_plug(self):
        pass

class MotorEletrico:
    def __init__(self):
        print('Motor elétrico criado')

class Veículo:
    def __init__(self):
        print('Veículo criado')
        self.motor = MotorCombustao()

class LunarRover(Vehicle):
    change_spark_plug = None
    def __init__(self):
        print('Veículo Lunar Criado')
        self.motor = MotorEletrico()

In [51]:
class MotorEletrico:
    def __init__(self):
        print('Motor elétrico criado')
    
    def __str__(self):
        return 'Eu sou um motor elétrico!'
    
class MotorCombustao:
    def __init__(self):
        print('Motor à combustão criado')
    
    def __str__(self):
        return 'Eu sou um motor à combustão!'

class Veiculo:
    def __init__(self, motor):
        self.motor = motor
    
    def show_motor(self):
        print(self.motor)

class Carro(Veiculo):
    def __init__(self):
        print('Carro criado')
        super().__init__(MotorCombustao())

class VeiculoLunar(Veiculo):
    def __init__(self):
        print('Veículo lunar criado')
        super().__init__(MotorEletrico())

In [52]:
car = Carro()
rover = VeiculoLunar()

Carro criado
Motor à combustão criado
Veículo lunar criado
Motor elétrico criado


In [54]:
car.show_motor()

Eu sou um motor à combustão!


In [55]:
rover.show_motor()

Eu sou um motor elétrico!


### Heranças Múltiplas e Mixins

> Work in progress

## Composição

A composição modela uma relação do tipo **"composto por"** (has a). Isto significa que uma classe compositora tem como atributos instâncias de outras classes (chamamos essas instâncias de _componente_).

Composição é uma forma de utilizar lógicas definidas nos componentes através de suas interfaces. Ainda, dado que apenas as interfaces são utilizadas, alterações no componente não impactam a classe compositora, como ocorre com herança (exemplo apresentado na seção [Alto Acoplamento](#alto-acoplamento), onde uma alteração que deveria impactar apenas um conjunto de classes derivadas, impactava todas, tornando necessário um grande esforço para adaptação).

In [59]:
import uuid

class Endereco:
    def __init__(self, rua, numero, cep):
        self.rua = rua
        self.numero = numero
        self.cep = cep
    
    def __str__(self):
        return f'Rua {self.rua}, {self.numero}. CEP: {self.cep}'

class PessoaFuncionaria:
    def __init__(self, nome, endereco):
        self.id = str(uuid.uuid4())
        self.nome = nome
        self.endereco = endereco

In [60]:
f = PessoaFuncionaria(
    nome='Carl Gauss',
    endereco=Endereco(
        rua='Street 1',
        numero='475',
        cep='12345',
    )
)

print(f.endereco)

Rua Street 1, 475. CEP: 12345


## Favorecer Composição Sobre Herança

A fim de evitar os problemas no uso de herança, muitas pessoas preferem modelar relacionamentos entre elementos da forma **"composto por"** (composição). Como dito na seção [Composição](#composição), tal estratégia nos ajuda a manter o código menos desacoplado, ao mesmo tempo em que temos a vantagem de reuso de código.

> Note, no entanto, que isso não significa que o código deve estar ser totalmente desacoplado! Afinal, um código sem acoplamento é um código onde os elementos não interagem entre si e, consequentemente, nenhuma ação é feita! Ainda, o uso de herança permanece válido quando usada de forma apropriada!

Para mais detalhes, veja os links abaixo:

- [Inheritance and Composition: A Python OOP Guide by Real Python](https://realpython.com/inheritance-composition-python/)
- [Why COMPOSITION is better than INHERITANCE - detailed Python example](https://www.youtube.com/watch?v=0mcP8ZpUR38)

## Conclusões

- Herança é uma ótima alternativa para reuso de código. Porém, devemos usar com cuidado visto que é fácil se perder.
- Favorecer o uso de composição sempre que possível.
- Classes abstratas são recomendáveis quando o objetivo é definir um "contrato" entre um conjunto de classes com comportamentos semelhantes.

## Referências

- [Python OOP Tutorial 4: Inheritance - Creating Subclasses by Corey Schafer](https://www.youtube.com/watch?v=RSl87lqOXDE)
- [OBJECT-ORIENTED PROGRAMMING AND INHERITANCE](https://inventwithpython.com/beyond/chapter16.html)
- [Object-Oriented Programming (OOP) in Python 3 by Real Python](https://realpython.com/python3-object-oriented-programming/)
- [Inheritance and Composition: A Python OOP Guide by Real Python](https://realpython.com/inheritance-composition-python/)